In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from Contrast_loss import ContrastLoss
from model import Model
device = torch.device("mps")

/Users/garrygeng/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/garrygeng/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch.__version__

'2.0.1'

In [4]:
model = Model()

In [5]:
def AC_loss(y_true, y_pred):

    x = y_pred[:, :, 1:] - y_pred[:, : , :-1]
    delta_x = x[:, :, :-2] ** 2

    length = torch.mean(torch.sqrt(delta_x + 1e-6), axis=2)


    c1 = torch.ones_like(y_true)
    c2 = torch.zeros_like(y_true)
    region_in = torch.abs(torch.mean(y_pred * ((y_true - c1) ** 2), axis=2))

    region_out = torch.abs(torch.mean((1 - y_pred) * ((y_true - c2) ** 2), axis=2))


    return 6 * length + (region_in + region_out)


def dice_metric(y_true, y_pred):
    intersection = torch.sum(y_pred * y_true)
    smooth = 0.0000001
    dice = (2. * intersection + smooth) / (torch.sum(y_true) + torch.sum(y_pred) + smooth)
    return dice

CE_loss = nn.BCELoss()
contrast_criterion = ContrastLoss()


In [6]:
X_train = np.load('data/X_train.npy')
X_val = np.load('data/X_val.npy')
y_seg_train = np.load('data/y_seg_train.npy')
y_seg_val = np.load('data/y_seg_val.npy')

X_train = X_train.reshape((X_train.shape[0], 1 ,  X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
y_seg_train = y_seg_train.reshape((y_seg_train.shape[0], 1, y_seg_train.shape[1]))
y_seg_val = y_seg_val.reshape((y_seg_val.shape[0], 1 ,  y_seg_val.shape[1]))



In [7]:
X_train = torch.FloatTensor(X_train)
X_val = torch.FloatTensor(X_val)
y_seg_train = torch.FloatTensor(y_seg_train)
y_seg_val = torch.FloatTensor(y_seg_val)

In [8]:
import torch.utils.data as Data
train_data = Data.TensorDataset(X_train,y_seg_train)
train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=64,
    shuffle=True,
)


val_data = Data.TensorDataset(X_val,y_seg_val)
val_loader = Data.DataLoader(
    dataset=val_data,
    batch_size=64,
    shuffle=True,
)

In [9]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
optimizer = optim.Adam(
    model.parameters(),
    lr=0.0005,
)





In [11]:
import math
bacth_len = math.ceil(X_train.shape[0]/64)

best_valid_dice = 0
model = model.to(device)


for epoch in range(5000):
    running_loss = 0.0
    model.train()
    for batch_idx, data in enumerate(train_loader, start=0):
        x, labels = data 
        x = x.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()                     
        outputs , proj_feature = model(x)['seg'] , model(x)['embed']  
   

        preds = outputs.clone()
        preds[preds <= 0.5] = 0
        preds[preds > 0.5] = 1


        # loss = CE_loss(outputs, labels)+0.1*contrast_criterion(proj_feature,labels,preds)  
        loss = torch.mean(AC_loss(labels, outputs))+ 0.1*contrast_criterion(proj_feature,labels,preds)  
  
        # loss.backward(loss.clone().detach())    
        loss.backward()                           
        optimizer.step()                          
        running_loss += torch.mean(loss).item()

    model.eval()
    with torch.no_grad():
          val_x = X_val.to(device)
          val_outputs = model(val_x)['seg']
          val_y = y_seg_val.to(device)  
  
          val_outputs = np.array(val_outputs.cpu().flatten()).astype(np.int8)
          val_outputs[val_outputs <= 0.5] = 0
          val_outputs[val_outputs > 0.5] = 1
          val_y = np.array(val_y.cpu().flatten()).astype(np.int8)
    
          intersection = np.sum(val_outputs * val_y)
          smooth = 0.00001
          dice = (2. * intersection + smooth) / (np.sum(val_y) + np.sum(val_outputs) + smooth)
          print('val_dice:',dice)

    if dice > best_valid_dice:
           best_valid_dice  = dice
           print("save model")
           import os
           os.makedirs("Save_Model", exist_ok=True)
           torch.save(model, 'Save_Model/model-2023-5-31-1') 
           torch.save(model.state_dict(), 'Save_Model/model_parameter-2023-5-31-1.pkl')

    print('epoch:{} batch_idx:{} loss:{}'
                  .format(epoch+1, batch_idx+1, running_loss/bacth_len))

    running_loss = 0.0 

val_dice: 2.490225863423806e-11
save model
epoch:1 batch_idx:49 loss:2.0628431646191343
val_dice: 2.490225863423806e-11
epoch:2 batch_idx:49 loss:1.774668457556744
val_dice: 2.490225863423806e-11
epoch:3 batch_idx:49 loss:1.5564794321449436
val_dice: 2.490225863423806e-11
epoch:4 batch_idx:49 loss:1.3189243832413031
val_dice: 2.490225863423806e-11
epoch:5 batch_idx:49 loss:1.2982174498694283
val_dice: 0.06418353859815439
save model
epoch:6 batch_idx:49 loss:1.0274345230083077
val_dice: 0.00012948306386428477
epoch:7 batch_idx:49 loss:1.1727326597486223
val_dice: 0.02827607548176585
epoch:8 batch_idx:49 loss:1.0163869614503822
val_dice: 0.07141039298641236
save model
epoch:9 batch_idx:49 loss:1.0651361626021716
val_dice: 0.08793244704763127
save model
epoch:10 batch_idx:49 loss:0.9553628807165184
val_dice: 0.09187676896172049
save model
epoch:11 batch_idx:49 loss:0.9292207846836168
val_dice: 0.11270842931569074
save model
epoch:12 batch_idx:49 loss:0.9408379732346048
val_dice: 0.1082994

RuntimeError: MPS backend out of memory (MPS allocated: 2.72 GB, other allocations: 5.16 GB, max allowed: 9.07 GB). Tried to allocate 1.26 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).